# Prototyping Trace Log API 

In [ ]:
#r "nuget:Microsoft.Diagnostics.Tracing.TraceEvent"
#r "nuget:XPlot.Plotly"
#r "nuget:XPlot.GoogleCharts"

Installed Packages Microsoft.Diagnostics.Tracing.TraceEvent, 2.0.74 XPlot.GoogleCharts, 3.0.1 XPlot.Plotly, 4.0.6

In [ ]:
// Open all the libraries.
open Microsoft.Diagnostics.Tracing;
open Microsoft.Diagnostics.Tracing.Etlx;
open Microsoft.Diagnostics.Tracing.Session;
open Microsoft.Diagnostics.Tracing.Parsers.Clr;

open XPlot.Plotly;
open XPlot.GoogleCharts;

In [ ]:
let ETL_FILEPATH = @"C:\Users\mukun\OneDrive\Documents\CallstackShmuff.etl\CallstackShmuff.etl" 

## Charting Allocation Events Using Trace Log

In [ ]:
let session = new TraceEventSession("TestSession", ETL_FILEPATH)

let traceLog = TraceLog.OpenOrConvert(ETL_FILEPATH)

let allocationAmountForDevenv =
    traceLog.Events
    |> Seq.filter(fun e -> e.ProcessName = "devenv" && e.EventName.Contains("GC/AllocationTick"))
    |> Seq.take 100
    |> Seq.map(fun e -> (e.TimeStampRelativeMSec, float(e.PayloadByName("AllocationAmount").ToString())))

let chart = 
    let options = Options( title = "Allocation Amount Over Time", 
                           vAxes = [| Axis(title = "Allocation Amount"); Axis(title = "Timestamp"); |] )

    allocationAmountForDevenv
    |> Chart.Line
    |> Chart.WithOptions options
    |> Chart.Show

## Call Stack Collection Using Trace Log

In [ ]:
open Microsoft.Diagnostics.Symbols;
open Microsoft.Diagnostics.Tracing;
open Microsoft.Diagnostics.Tracing.Etlx;
open Microsoft.Diagnostics.Tracing.Parsers;

In [ ]:
let session = new TraceEventSession("TestSession", ETL_FILEPATH)

let traceLog = TraceLog.OpenOrConvert(ETL_FILEPATH)

let loadSymbols : unit = 
    use symbolReader = new SymbolReader(TextWriter.Null, SymbolPath.SymbolPathFromEnvironment)
    traceLog.Processes
    |> Seq.filter(fun p -> p.Name = "GCRealTimeMon.exe")
    |> Seq.iter(fun proc -> ( 
        proc.LoadedModules |> Seq.where (fun m -> not (isNull m.ModuleFile))
                        |> Seq.iter (fun m -> traceLog.CodeAddresses.LookupSymbolsForModule(symbolReader, m.ModuleFile))

    ))

In [ ]:
 let processCallStack (callStack : TraceCallStack) : unit =

    use symbolReader = new SymbolReader(TextWriter.Null, SymbolPath.SymbolPathFromEnvironment)

    let printStackFrame (callStack : TraceCallStack) : unit =
        traceLog.CodeAddresses.LookupSymbolsForModule(symbolReader, callStack.CodeAddress.ModuleFile)
        printfn "%s!%s#%s" callStack.CodeAddress.ModuleName callStack.CodeAddress.FullMethodName (callStack.CodeAddress.GetSourceLine(symbolReader).LineNumber.ToString())

    let rec processFrame (callStack : TraceCallStack) : unit =
        if isNull callStack then
            ()
        else
            printStackFrame callStack
            processFrame callStack.Caller

    processFrame callStack

let printGCAllocStacksForGCRealTimeMon : unit =
    traceLog.Events
    |> Seq.filter(fun e -> e.ProcessName = "GCRealTimeMon" && e.EventName = "GC/AllocationTick")
    |> Seq.take 1 
    |> Seq.iter(fun e ->( 
            printfn "\n"; processCallStack (e.CallStack())))

Error: input.fsx (7,17)-(7,27) typecheck error Type mismatch. Expecting a
    'string -> string -> unit'    
but given a
    'string -> string -> string -> unit'    
The type 'unit' does not match the type 'string -> unit'